In [1]:
%run helper/setup_notebook.ipynb import display table

Successfully connected to leetcode50 database.


In [2]:
tables = ['Prices', 'UnitsSold']
for table in tables:
    display_table(table)

+------------+------------+------------+-------+
| product_id | start_date |  end_date  | price |
+------------+------------+------------+-------+
|     1      | 2019-02-17 | 2019-02-28 |   5   |
|     1      | 2019-03-01 | 2019-03-22 |   20  |
|     2      | 2019-02-01 | 2019-02-20 |   15  |
|     2      | 2019-02-21 | 2019-03-31 |   30  |
+------------+------------+------------+-------+
+------------+---------------+-------+
| product_id | purchase_date | units |
+------------+---------------+-------+
|     1      |   2019-02-25  |  100  |
|     1      |   2019-03-01  |   15  |
|     2      |   2019-02-10  |  200  |
|     2      |   2019-03-22  |   30  |
+------------+---------------+-------+


### Write an SQL query to find the average selling price for each product. average_price should be rounded to 2 decimal places.

```
+------------+---------------+
| product_id | average_price |
+------------+---------------+
| 1          | 6.96          |
| 2          | 16.96         |
+------------+---------------+
Explanation: 
Average selling price = Total Price of Product / Number of products sold.
Average selling price for product 1 = ((100 * 5) + (15 * 20)) / 115 = 6.96
Average selling price for product 2 = ((200 * 15) + (30 * 30)) / 230 = 16.96
```

In [3]:
%%sql 

SELECT 
    p.product_id, 
    ROUND((AVG(price * units) / avg(units)), 2) AS average_price
FROM prices p 
JOIN UnitsSold u ON u.product_id = p.product_id
WHERE purchase_date BETWEEN start_date AND end_date
GROUP BY product_id 

product_id,average_price
1,6.96
2,16.96


# Using Pandas

In [4]:
import pandas as pd 

In [5]:
prices_query = %sql SELECT * FROM Prices # type: ignore
units_query = %sql SELECT * FROM UnitsSold # type: ignore
prices_df = prices_query.DataFrame()
units_df = units_query.DataFrame()
display(prices_df, units_df)

,product_id,start_date,end_date,price
0,1,2019-02-17,2019-02-28,5
1,1,2019-03-01,2019-03-22,20
2,2,2019-02-01,2019-02-20,15
3,2,2019-02-21,2019-03-31,30


,product_id,purchase_date,units
0,1,2019-02-25,100
1,1,2019-03-01,15
2,2,2019-02-10,200
3,2,2019-03-22,30


In [14]:
prices_df.merge(units_df, on='product_id', how='outer')

,product_id,start_date,end_date,price,purchase_date,units
0,1,2019-02-17,2019-02-28,5,2019-02-25,100
1,1,2019-02-17,2019-02-28,5,2019-03-01,15
2,1,2019-03-01,2019-03-22,20,2019-02-25,100
3,1,2019-03-01,2019-03-22,20,2019-03-01,15
4,2,2019-02-01,2019-02-20,15,2019-02-10,200
5,2,2019-02-01,2019-02-20,15,2019-03-22,30
6,2,2019-02-21,2019-03-31,30,2019-02-10,200
7,2,2019-02-21,2019-03-31,30,2019-03-22,30


In [6]:
merged_df = prices_df.merge(units_df) # both 'on' and 'how' are optional
merged_df

,product_id,start_date,end_date,price,purchase_date,units
0,1,2019-02-17,2019-02-28,5,2019-02-25,100
1,1,2019-02-17,2019-02-28,5,2019-03-01,15
2,1,2019-03-01,2019-03-22,20,2019-02-25,100
3,1,2019-03-01,2019-03-22,20,2019-03-01,15
4,2,2019-02-01,2019-02-20,15,2019-02-10,200
5,2,2019-02-01,2019-02-20,15,2019-03-22,30
6,2,2019-02-21,2019-03-31,30,2019-02-10,200
7,2,2019-02-21,2019-03-31,30,2019-03-22,30


In [7]:
merged_df.groupby('product_id').agg({'price':'mean'})

,price
product_id,
1,12.5
2,22.5


In [15]:
merged_df.query("start_date <= purchase_date <= end_date").groupby('product_id').agg({'price': 'mean'})

,price
product_id,
1,12.5
2,22.5


In [16]:
merged_df.query("start_date <= purchase_date <= end_date").groupby('product_id').agg({'units': 'mean'})

,units
product_id,
1,57.5
2,115.0


In [27]:
merged_df.query("start_date <= purchase_date <= end_date").groupby('product_id').agg({'units': 'sum'})

,units
product_id,
1,115
2,230


## filtering

In [18]:
merged_df[(merged_df['purchase_date'] >= merged_df['start_date'])]

,product_id,start_date,end_date,price,purchase_date,units
0,1,2019-02-17,2019-02-28,5,2019-02-25,100
1,1,2019-02-17,2019-02-28,5,2019-03-01,15
3,1,2019-03-01,2019-03-22,20,2019-03-01,15
4,2,2019-02-01,2019-02-20,15,2019-02-10,200
5,2,2019-02-01,2019-02-20,15,2019-03-22,30
7,2,2019-02-21,2019-03-31,30,2019-03-22,30


In [19]:
filtered_df = merged_df[(merged_df['purchase_date'] >= merged_df['start_date']) & 
                        (merged_df['purchase_date'] <= merged_df['end_date'])]
filtered_df

,product_id,start_date,end_date,price,purchase_date,units
0,1,2019-02-17,2019-02-28,5,2019-02-25,100
3,1,2019-03-01,2019-03-22,20,2019-03-01,15
4,2,2019-02-01,2019-02-20,15,2019-02-10,200
7,2,2019-02-21,2019-03-31,30,2019-03-22,30


In [23]:
filtered_df.groupby('product_id').apply(lambda x: (x['price'] * x['units']).mean())

product_id
1     400.0
2    1950.0
dtype: float64

In [24]:
filtered_df.groupby('product_id') \
    .apply(lambda x: (x['price'] * x['units']).mean() / x['units'].mean())

product_id
1     6.956522
2    16.956522
dtype: float64

In [25]:
filtered_df.groupby('product_id') \
    .apply(lambda x: round((x['price'] * x['units']).mean() / x['units'].mean(), 2))

product_id
1     6.96
2    16.96
dtype: float64

In [26]:
grouped_df = filtered_df.groupby('product_id') \
    .apply(lambda x: round((x['price'] * x['units']).mean() / x['units'].mean(), 2)) \
    .reset_index(name='average_price')
grouped_df

,product_id,average_price
0,1,6.96
1,2,16.96
